In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import median_absolute_error

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy

import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [209]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [210]:
df_pca1_train = pd.read_pickle('../feats/pca.pkl')
df_pca1_test = pd.read_pickle('../feats/pca_test.pkl')
df_train = pd.concat([df_train, df_pca1_train], axis=1)
df_test = pd.concat([df_test, df_pca1_test], axis=1)

In [211]:
df_pca2_train = pd.read_pickle('../feats/pca2_train.pkl')
df_pca2_test = pd.read_pickle('../feats/pca2_test.pkl')
df_train = pd.merge(df_train, df_pca2_train, on='index')
df_test = pd.merge(df_test, df_pca2_test, on='index')

In [212]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [213]:
df_tsne_train = pd.read_pickle('../feats/tsne.pkl')
df_tsne_test = pd.read_pickle('../feats/tsne_test.pkl')
df_train = pd.merge(df_train, df_tsne_train, on='index')
df_test = pd.merge(df_test, df_tsne_test, on='index')

In [214]:
df_vae_train = pd.read_pickle('../feats/vae.pkl')
df_vae_test = pd.read_pickle('../feats/vae_test.pkl')
df_train = pd.merge(df_train, df_vae_train, on='index')
df_test = pd.merge(df_test, df_vae_test, on='index')

In [215]:
db_catboost = DFDB('../trial/catboost.pkl', auto_commit=False)
df_trial_catboost = db_catboost.select()
df_trial_catboost['kfold'] = df_trial_catboost['param'].apply(lambda x: x['kfold'])

In [252]:
df_trial_catboost.loc[2231]['param']['columns']

['mfcc_3_rolling_std_mean',
 'max_9',
 'q25_roll_std_100',
 'max_to_min',
 'max_to_min_5',
 'iqr_6',
 'q05_roll_std_1000',
 'q05_roll_std_100',
 'abs_max_4',
 'abs_max_1',
 'spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'q01_2',
 'abs_q01_4',
 'max_to_min_diff_5',
 'q05_5',
 'abs_q25_5',
 'abs_max_2',
 'min__roll_std',
 'median__roll_std',
 'abs_max_7',
 '5000peak_peak_amp_max_',
 'abs_q75_6']

In [216]:
db_xgbm = DFDB('../trial/xgbm.pkl', auto_commit=False)
df_trial_xgbm = db_xgbm.select()
df_trial_xgbm['kfold'] = df_trial_xgbm['param'].apply(lambda x: x['kfold'])

In [217]:
db_lgbm = DFDB('../trial/lgbm.pkl', auto_commit=False)
df_trial_lgbm = db_lgbm.select()
df_trial_lgbm['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])

In [218]:
db_lgbm4 = DFDB('../trial/lgbm4.pkl', auto_commit=False)
df_trial_lgbm4 = db_lgbm4.select()
df_trial_lgbm4['kfold'] = df_trial_lgbm4['param'].apply(lambda x: x['kfold'])

In [219]:
db_randomforest = DFDB('../trial/randomforest.pkl', auto_commit=False)
df_trial_randomforest = db_randomforest.select()
df_trial_randomforest['kfold'] = df_trial_randomforest['param'].apply(lambda x: x['kfold'])

In [220]:
db_extratrees = DFDB('../trial/extratrees.pkl', auto_commit=False)
df_trial_extratrees = db_extratrees.select()
df_trial_extratrees['kfold'] = df_trial_extratrees['param'].apply(lambda x: x['kfold'])

In [221]:
db_gradientboosting = DFDB('../trial/gradientboosting.pkl', auto_commit=False)
df_trial_gradientboosting = db_gradientboosting.select()
df_trial_gradientboosting['kfold'] = df_trial_gradientboosting['param'].apply(lambda x: x['kfold'])

In [222]:
db_knn = DFDB('../trial/knn.pkl', auto_commit=False)
df_trial_knn = db_knn.select()
df_trial_knn['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [223]:
db_svr = DFDB('../trial/svr.pkl', auto_commit=False)
df_trial_svr = db_svr.select()
df_trial_svr['kfold'] = df_trial_svr['param'].apply(lambda x: x['kfold'])

In [224]:
db_fm = DFDB('../trial/fm.pkl', auto_commit=False)
df_trial_fm = db_fm.select()
df_trial_fm['kfold'] = df_trial_fm['param'].apply(lambda x: x['kfold'])

In [225]:
db_lasso = DFDB('../trial/lasso.pkl', auto_commit=False)
df_trial_lasso = db_lasso.select()
df_trial_lasso['kfold'] = df_trial_lasso['param'].apply(lambda x: x['kfold'])

In [226]:
db_ridge = DFDB('../trial/ridge.pkl', auto_commit=False)
df_trial_ridge = db_ridge.select()
df_trial_ridge['kfold'] = df_trial_ridge['param'].apply(lambda x: x['kfold'])

In [227]:
db_kerasmlp = DFDB('../trial/kerasmlp.pkl', auto_commit=False)
df_trial_mlp = db_kerasmlp.select()
df_trial_mlp['kfold'] = df_trial_mlp['param'].apply(lambda x: x['kfold'])

In [228]:
db_frgf = DFDB('../trial/frgf.pkl', auto_commit=False)
df_trial_frgf = db_frgf.select()
df_trial_frgf['kfold'] = df_trial_frgf['param'].apply(lambda x: x['kfold'])

In [229]:
db_stacknet_pca = DFDB('../trial/stacknet_pca.pkl', auto_commit=False)
df_trial_stacknet_pca = db_stacknet_pca.select()
df_trial_stacknet_pca['kfold'] = df_trial_stacknet_pca['param'].apply(lambda x: x['kfold'])

In [320]:
db_mystack = DFDB('../trial/mystack.pkl', auto_commit=False)
df_trial_mystack = db_mystack.select()
df_trial_mystack['kfold'] = df_trial_mystack['param'].apply(lambda x: x['kfold'])

In [319]:
db_stacknet_pca_mfcc = DFDB('../trial/stacknet_pca_mfcc.pkl', auto_commit=False)
df_trial_stacknet_pca_mfcc = db_stacknet_pca_mfcc.select()
df_trial_stacknet_pca_mfcc['kfold'] = df_trial_frgf['param'].apply(lambda x: x['kfold'])

In [232]:
trial_lst = [
    df_trial_catboost.loc[452], 
    df_trial_xgbm.loc[1172], 
    df_trial_lgbm.loc[2156],
    df_trial_gradientboosting.loc[306],
    df_trial_randomforest.loc[297],
    df_trial_extratrees.loc[459],
    df_trial_knn.loc[17], 
    df_trial_svr.loc[7],
    df_trial_fm.loc[313],
    df_trial_lasso.loc[8], 
    df_trial_ridge.loc[15],
    df_trial_mlp.loc[13], 
    df_trial_frgf.loc[200],
    
    df_trial_catboost.loc[2231], #g3
    df_trial_catboost.loc[2309], #s3
    df_trial_lgbm4.loc[805], #g3
    df_trial_lgbm4.loc[917], #s3
    df_trial_xgbm.loc[2131], #g3
    df_trial_xgbm.loc[2379], #s3
    df_trial_gradientboosting.loc[1280], #g3
    df_trial_gradientboosting.loc[1637], #s3
    df_trial_ridge.loc[1525], #g3
    df_trial_ridge.loc[1826], #s3
    df_trial_lasso.loc[2277], #g3
    df_trial_lasso.loc[2517], #s3
    df_trial_knn.loc[1383], #g3
    df_trial_knn.loc[1422], #s3
    df_trial_svr.loc[918], #g3
    df_trial_mlp.loc[264], #g3
    df_trial_mlp.loc[291], #s3
    
]
name_lst = [
    'cb452', 
    'xgbm1172', 
    'lgbm2156',
    'gbm306',
    'rf297',
    'et459',
    'knn17',
    'svr7', 
    'fm313',
    'lasso8',
    'ridge15',
    'mlp13',
    'frgf200',
    
    'cb2231',
    'cb2309',
    'lgbm805',
    'lgbm917',
    'xgbm2131',
    'xgbm2379',
    'gradientboosting1280',
    'gradientboosting1637',
    'ridge1525',
    'ridge1826',
    'lasso2277',
    'lasso2517',
    'knn1383',
    'knn1422',
    'svr918',
    'mlp264',
    'mlp291',
]

In [233]:
for r in trial_lst:
    print(r['param']['algorithm']['cls'], r['param']['algorithm']['init'], r['param']['kfold']['type'], r['param']['kfold']['n_splits'], len(r['param']['columns']), r['val_mae'], r['mae_diff'])
    print()

cb.CatBoostRegressor {'num_trees': 929, 'depth': 5, 'learning_rate': 0.01571023413441655, 'l2_leaf_reg': 0.49142786214498846, 'bagging_temperature': 0.6008299770689625, 'random_strength': 0.7930166632962454, 'random_state': 1569} stratified 3 30 1.9005345457826346 0.04705627937966006

xgb.XGBRegressor {'max_depth': 3, 'max_bin': 77, 'eta': 0.28071904276671816, 'colsample_bytree': 0.9102857670808742, 'min_child_weight': 271, 'n_estimators': 113, 'subsample': 0.6831182550379268, 'reg_lambda': 0.812561328581548, 'reg_alpha': 0.007362369490410914, 'n_jobs': 16} stratified 3 25 1.9655602276971649 0.040781720685815426

lgb.LGBMRegressor {'learning_rate': 0.19329183957823715, 'feature_fraction': 0.7363782645537933, 'bagging_fraction': 0.8585309819625903, 'min_data_in_leaf': 351, 'lambda_l1': 95.63411922439124, 'lambda_l2': 48.949784919880365, 'max_bin': 25, 'num_leaves': 8, 'random_state': 5040, 'n_jobs': 32} stratified 3 25 1.9528967095257748 0.04172569264886117

GradientBoostingRegressor {'

In [234]:
df_train_oof = pd.DataFrame()
df_test_oof = pd.DataFrame()
for df_, name_ in zip(trial_lst, name_lst):
    try:
        df_test_oof_i = df_['df_test_pred']
        df_test_oof[name_] = np.mean(df_test_oof_i.drop(columns=['index']).values, axis=1)
        
        df_train_oof_i = df_['df_valid_pred']
        df_train_oof[name_] = df_train_oof_i['predict']
        
    except Exception as e:
        print(name_, e.__str__())

df_train_oof['index'] = df_train_oof_i['index']
df_test_oof['index'] = df_test_oof_i['index']
df_train = pd.merge(df_train, df_train_oof, on='index')
df_test = pd.merge(df_test, df_test_oof, on='index')

In [29]:
# mytrial =[]
# columns= param['columns']
# df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train[columns+['y','index','group','label']], param, df_test=df_test[columns+['index']], trial=mytrial, is_output_feature_importance=False, remark='rebuild mystacknet_pca_469')

In [56]:
df_trial_stacknet_pca_mfcc[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].loc[[1198]]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1198,2019-06-01 03:38:44.365785,15,1.745461,0.000527,1.837021,0.003617,0.09156


In [137]:
stacknet1198 =  ['mlp13',
  'q01_roll_std_100',
  'mfcc_accelerate_8_variance',
  'abs_q75_2',
  'kle',
  'xgbm1172',
  'median__roll_std',
  'svr7',
  'ridge15',
  'lgbm2156',
  'spkt_welch_density__coeff_3',
  'gbm306',
  'knn17',
  'lasso8',
  'cb452']

In [237]:
pca_columns = ['5000no_zero_crossing_mean__pca0',"value_count{'value': 1}_pca0"]
new_oof_columns = ['cb2231',
    'cb2309',
    'lgbm805',
    'lgbm917',
    'xgbm2131',
    'xgbm2379',
    'gradientboosting1280',
    'gradientboosting1637',
    'ridge1525',
    'ridge1826',
    'lasso2277',
    'lasso2517',
    'knn1383',
    'knn1422',
    'svr918',
    'mlp264',
    'mlp291']

In [253]:
catboost2231 = ['mfcc_3_rolling_std_mean',
 'max_9',
 'q25_roll_std_100',
 'max_to_min',
 'max_to_min_5',
 'iqr_6',
 'q05_roll_std_1000',
 'q05_roll_std_100',
 'abs_max_4',
 'abs_max_1',
 'spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'q01_2',
 'abs_q01_4',
 'max_to_min_diff_5',
 'q05_5',
 'abs_q25_5',
 'abs_max_2',
 'min__roll_std',
 'median__roll_std',
 'abs_max_7',
 '5000peak_peak_amp_max_',
 'abs_q75_6']

In [256]:
mytrial = []
param = {'columns': [],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 718,
   'depth': 3,
   'learning_rate': 0.03120323743802602,
   'l2_leaf_reg': 0.1761783064404861,
   'bagging_temperature': 0.884063752227235,
   'random_strength': 0.38441807259601113,
   'random_state': 1589,
   'logging_level': 'Silent'},
  'fit': {}}}
param['columns'] = stacknet1198  + pca_columns + new_oof_columns + ['kle']
df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param, df_test=df_test, trial=mytrial, is_output_feature_importance=False, remark='init final2')

In [257]:
score = df_his.valid.mean()
EP.width_frist_rfe(df_train, param, mytrial, score, df_test=df_test, remark='wf final2')

In [258]:
for trial_i in mytrial:
    db_stacknet_pca_mfcc.insert(trial_i)

In [259]:
df_trial = db_stacknet_pca_mfcc.select()
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [297]:
df_trial[(df_trial['remark']=='wf final')][['datetime','nfeatures', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark','algorithm-init']].sort_values(by=['val_mae'], ascending=True).head()

,datetime,nfeatures,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark,algorithm-init
2990,2019-06-03 10:31:53.913283,31,wf final,1.692345,0.000378,1.779966,0.003456,0.087621,wf final,"{'num_trees': 718, 'depth': 3, 'learning_rate': 0.03120323743802602, 'l2_leaf_reg': 0.1761783064404861, 'bagging_temperature': 0.884063752227235, 'random_strength': 0.38441807259601113, 'random_state': 1589, 'logging_level': 'Silent'}"
3020,2019-06-03 10:40:03.082196,30,wf final,1.692353,0.000396,1.780503,0.003223,0.088150,wf final,"{'num_trees': 718, 'depth': 3, 'learning_rate': 0.03120323743802602, 'l2_leaf_reg': 0.1761783064404861, 'bagging_temperature': 0.884063752227235, 'random_strength': 0.38441807259601113, 'random_state': 1589, 'logging_level': 'Silent'}"
3038,2019-06-03 10:44:56.005717,30,wf final,1.692105,0.000384,1.780827,0.003224,0.088722,wf final,"{'num_trees': 718, 'depth': 3, 'learning_rate': 0.03120323743802602, 'l2_leaf_reg': 0.1761783064404861, 'bagging_temperature': 0.884063752227235, 'random_strength': 0.38441807259601113, 'random_state': 1589, 'logging_level': 'Silent'}"
2978,2019-06-03 10:28:38.859507,31,wf final,1.692098,0.000418,1.780855,0.003296,0.088757,wf final,"{'num_trees': 718, 'depth': 3, 'learning_rate': 0.03120323743802602, 'l2_leaf_reg': 0.1761783064404861, 'bagging_temperature': 0.884063752227235, 'random_strength': 0.38441807259601113, 'random_state': 1589, 'logging_level': 'Silent'}"
2988,2019-06-03 10:31:20.645230,31,wf final,1.692545,0.000376,1.780990,0.003588,0.088445,wf final,"{'num_trees': 718, 'depth': 3, 'learning_rate': 0.03120323743802602, 'l2_leaf_reg': 0.1761783064404861, 'bagging_temperature': 0.884063752227235, 'random_strength': 0.38441807259601113, 'random_state': 1589, 'logging_level': 'Silent'}"


In [308]:
df_trial.loc[2474]['param']

{'algorithm': {'cls': 'Lasso',
  'fit': {},
  'init': {'alpha': 0.006856644356930668,
   'fit_intercept': True,
   'normalize': False,
   'precompute': False,
   'copy_X': True,
   'max_iter': 3548,
   'tol': 0.0005668918092994284,
   'warm_start': False,
   'positive': False,
   'random_state': 3664,
   'selection': 'cyclic'}},
 'columns': ['27', '356', '452', '722', '1198', '1196', '1448', '1798'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}}}

In [254]:
db_stacknet_pca_mfcc.commit()

In [361]:
df_train_oof2 = pd.DataFrame()
df_test_oof2 = pd.DataFrame()

In [372]:
df_trial_stacknet_pca_mfcc.loc[[27, 452, 722, 1198, 1196]][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','kfold']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,kfold
27,2019-05-30 07:28:29.059867,tune pca 3,53,1.811007,0.000520,1.908997,0.003170,0.097989,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}"
452,2019-05-31 00:32:42.349643,tune 428,26,1.719680,0.000033,1.767340,0.000227,0.047659,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}"
722,2019-05-31 14:16:44.465849,"try add tsne, vae",27,1.780584,0.000391,1.897044,0.003055,0.116460,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}"
1198,2019-06-01 03:38:44.365785,tune 941 by group,15,1.745461,0.000527,1.837021,0.003617,0.091560,NaN
1196,2019-06-01 03:37:49.017526,tune 941 by group,15,1.747140,0.000578,1.836753,0.003168,0.089613,NaN


In [373]:
#df_trial_stacknet_pca_mfcc
for idx in [27, 452, 722, 1198, 1196]:
    try:
        
        train_y_valid_pred = df_trial_stacknet_pca_mfcc.loc[idx].df_valid_pred['predict'].values
        train_index = df_trial_stacknet_pca_mfcc.loc[idx].df_valid_pred['index'].values
        
        df_train_oof2['{}'.format(idx)] = train_y_valid_pred
        
        test_y_valid_pred = df_trial_stacknet_pca_mfcc.loc[idx].df_test_pred.drop(columns='index').mean(axis=1)
        test_index = df_trial_stacknet_pca_mfcc.loc[idx].df_test_pred['index'].values
        
        df_test_oof2['{}'.format(idx)] = test_y_valid_pred
        
    except Exception as e:
        print(e.__str__())

In [374]:
df_trial_stacknet_pca.loc[[469]][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','kfold']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,kfold
469,2019-05-27 10:24:12.606868,tune 280,47,1.730152,0.000015,1.778465,0.000249,0.048313,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"


In [375]:
#df_trial_stacknet_pca
for idx in [469]:
    try:
        
        train_y_valid_pred = df_trial_stacknet_pca.loc[idx].df_valid_pred['predict'].values
        train_index = df_trial_stacknet_pca.loc[idx].df_valid_pred['index'].values
        
        df_train_oof2['{}'.format(idx)] = train_y_valid_pred
        
        test_y_valid_pred = df_trial_stacknet_pca.loc[idx].df_test_pred.drop(columns='index').mean(axis=1)
        test_index = df_trial_stacknet_pca.loc[idx].df_test_pred['index'].values
        
        df_test_oof2['{}'.format(idx)] = test_y_valid_pred
        
    except Exception as e:
        print(e.__str__())

In [376]:
df_trial_mystack.loc[[562]][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','kfold']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,kfold
562,2019-05-19 20:38:58.981647,"add mlp,rgf tune stratified",43,1.734277,0.000014,1.784253,0.000267,0.049976,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"


In [377]:

#df_trial_mystack
for idx in [562]:
    try:
        
        train_y_valid_pred = df_trial_mystack.loc[idx].df_valid_pred['predict'].values
        train_index = df_trial_mystack.loc[idx].df_valid_pred['index'].values
        
        df_train_oof2['{}'.format(idx)] = train_y_valid_pred
        
        test_y_valid_pred = df_trial_mystack.loc[idx].df_test_pred.drop(columns='index').mean(axis=1)
        test_index = df_trial_mystack.loc[idx].df_test_pred['index'].values
        
        df_test_oof2['{}'.format(idx)] = test_y_valid_pred
        
    except Exception as e:
        print(e.__str__())

df_train_oof2['index'] = train_index
df_test_oof2['index'] = test_index

df_train_oof2['y'] = df_train['y']
df_train_oof2['label'] = df_train['label']
df_train_oof2['group'] = df_train['group']

In [378]:
df_train_oof2.head()

,27,452,722,1198,1196,469,562,index,y,label,group
0,5.195106,5.083498,5.396489,5.133132,5.000723,5.066370,5.087081,0,1.430797,1,1
1,5.875107,5.484911,5.892997,5.846838,5.715867,5.550073,5.690052,18750,1.425499,1,1
2,5.815038,5.466238,5.687898,5.913148,5.693465,5.721548,5.833274,37500,1.421296,1,1
3,5.674090,5.349848,5.714417,5.430884,5.541298,5.644359,5.493807,56250,1.415898,1,1
4,5.126699,4.856063,5.120558,5.103257,4.894409,5.249961,5.325288,75000,1.411696,1,1


In [379]:
df_test_oof2.head()

,27,452,722,1198,1196,469,562,index
0,3.455730,3.145287,3.214517,3.278950,3.171119,3.415621,3.451488,seg_00030f
1,5.724339,5.605531,5.837896,5.532883,5.917559,5.510218,5.683240,seg_0012b5
2,5.156434,5.352233,4.954962,5.277227,4.817586,5.450124,5.502084,seg_00184e
3,8.996977,9.391692,9.335360,9.090549,9.202680,9.528377,9.441776,seg_003339
4,7.489925,7.423252,7.664202,7.557975,7.874814,7.456676,7.711720,seg_0042cc


In [380]:
mytrial = []
#  tune hypterparameters
def objective(trial):

#     alpha = trial.suggest_int('alpha', 10, 1000)
#     tol = trial.suggest_uniform('tol', 0.0001, 0.01)
#     random_state = trial.suggest_int('random_state', 1, 9999)
        
#     args={'algorithm': {'cls': 'Ridge',
#       'fit': {},
#       'init': {'alpha': alpha,
#        'fit_intercept': True,
#        'normalize': False,
#        'copy_X': True,
#        'max_iter': None,
#        'tol': tol,
#        'solver': 'auto',
#        'random_state': random_state}},
#      'columns': df_train_oof2.columns.drop(['index', 'y', 'label', 'group']).tolist(),
#      'kfold': {'n_splits': 3,
#       'random_state': 1985,
#       'shuffle': True,
#       'type': 'group'},#stratified
#      'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    
    alpha = trial.suggest_uniform('alpha', 0.001, 10)
    tol = trial.suggest_uniform('tol', 0.00001, 0.001)
    max_iter = trial.suggest_int('max_iter', 500, 5000)
    random_state = trial.suggest_int('random_state', 1, 9999)
    
        
    args={'algorithm': {'cls': 'Lasso',
      'fit': {},
      'init': {'alpha': alpha,
       'fit_intercept': True,
       'normalize': False,
       'precompute': False,
       'copy_X': True,
       'max_iter': max_iter,
       'tol': tol,
       'warm_start': False,
       'positive': False,
       'random_state': random_state,
       'selection': 'cyclic'}},
     'columns': df_train_oof2.columns.drop(['index', 'y', 'label', 'group']).tolist(),
     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'group'},
     'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train_oof2, args, df_test = df_test_oof2, trial=mytrial, remark='tune bb')
    val_mae_mean = np.mean(df_his.valid)

    return  val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-06-03 13:31:28,110] Finished a trial resulted in value: 3.038739137437686. Current best value is 3.038739137437686 with parameters: {'alpha': 5.8215483187212165, 'tol': 0.0008709018016972637, 'max_iter': 4634, 'random_state': 2471}.
[I 2019-06-03 13:31:28,178] Finished a trial resulted in value: 3.0246436063757955. Current best value is 3.0246436063757955 with parameters: {'alpha': 2.8785839610464126, 'tol': 0.00029346843496172513, 'max_iter': 3734, 'random_state': 6481}.
[I 2019-06-03 13:31:28,244] Finished a trial resulted in value: 3.038739137437686. Current best value is 3.0246436063757955 with parameters: {'alpha': 2.8785839610464126, 'tol': 0.00029346843496172513, 'max_iter': 3734, 'random_state': 6481}.
[I 2019-06-03 13:31:28,330] Finished a trial resulted in value: 1.8978874365269685. Current best value is 1.8978874365269685 with parameters: {'alpha': 0.6742259489661055, 'tol': 0.00031825395434439545, 'max_iter': 765, 'random_state': 4834}.
[I 2019-06-03 13:31:28,408] F

[I 2019-06-03 13:31:31,161] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7711803391322407 with parameters: {'alpha': 0.07270730183227567, 'tol': 9.430004835330553e-05, 'max_iter': 4971, 'random_state': 1699}.
[I 2019-06-03 13:31:31,258] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7711803391322407 with parameters: {'alpha': 0.07270730183227567, 'tol': 9.430004835330553e-05, 'max_iter': 4971, 'random_state': 1699}.
[I 2019-06-03 13:31:31,358] Finished a trial resulted in value: 2.598578431576393. Current best value is 1.7711803391322407 with parameters: {'alpha': 0.07270730183227567, 'tol': 9.430004835330553e-05, 'max_iter': 4971, 'random_state': 1699}.
[I 2019-06-03 13:31:31,441] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7711803391322407 with parameters: {'alpha': 0.07270730183227567, 'tol': 9.430004835330553e-05, 'max_iter': 4971, 'random_state': 1699}.
[I 2019-06-03 13:31:31,538] 

[I 2019-06-03 13:31:36,920] Finished a trial resulted in value: 1.7527009492512329. Current best value is 1.7498836397412267 with parameters: {'alpha': 0.008892492301427746, 'tol': 9.262991788995673e-05, 'max_iter': 4853, 'random_state': 2680}.
[I 2019-06-03 13:31:37,047] Finished a trial resulted in value: 2.1743453256454797. Current best value is 1.7498836397412267 with parameters: {'alpha': 0.008892492301427746, 'tol': 9.262991788995673e-05, 'max_iter': 4853, 'random_state': 2680}.
[I 2019-06-03 13:31:37,177] Finished a trial resulted in value: 1.8312358331546605. Current best value is 1.7498836397412267 with parameters: {'alpha': 0.008892492301427746, 'tol': 9.262991788995673e-05, 'max_iter': 4853, 'random_state': 2680}.
[I 2019-06-03 13:31:37,305] Finished a trial resulted in value: 1.9576973496804504. Current best value is 1.7498836397412267 with parameters: {'alpha': 0.008892492301427746, 'tol': 9.262991788995673e-05, 'max_iter': 4853, 'random_state': 2680}.
[I 2019-06-03 13:31:

[I 2019-06-03 13:31:42,851] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:43,007] Finished a trial resulted in value: 1.7928525617842663. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:43,148] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:43,286] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31

[I 2019-06-03 13:31:48,802] Finished a trial resulted in value: 2.33974313274393. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:48,965] Finished a trial resulted in value: 2.1002101748508566. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:49,131] Finished a trial resulted in value: 2.951385726101521. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:49,283] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:

[I 2019-06-03 13:31:58,025] Finished a trial resulted in value: 2.1214146371199702. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:58,196] Finished a trial resulted in value: 2.6212626896162523. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:58,363] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:31:58,539] Finished a trial resulted in value: 1.973321634798875. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:3

[I 2019-06-03 13:32:04,251] Finished a trial resulted in value: 3.038739137437686. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.
[I 2019-06-03 13:32:04,448] Finished a trial resulted in value: 1.8609910189268728. Current best value is 1.7495139088294736 with parameters: {'alpha': 0.006781074204011127, 'tol': 0.00015014412886412944, 'max_iter': 3588, 'random_state': 4850}.


In [381]:
df_trial_oof2 = pd.DataFrame(mytrial)
df_trial_oof2['algorithm-cls'] = df_trial_oof2['param'].apply(lambda x : x['algorithm']['cls'])

In [383]:
df_trial_oof2.sort_values(by=['val_mae'], ascending=True).head(5)[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
76,2019-06-03 13:31:39.276734,tune bb,7,1.740072,0.000848,1.749514,0.002435,0.009442
163,2019-06-03 13:31:57.620958,tune bb,7,1.740186,0.000848,1.749520,0.002406,0.009334
114,2019-06-03 13:31:45.611974,tune bb,7,1.740942,0.000838,1.749690,0.002264,0.008748
65,2019-06-03 13:31:35.918290,tune bb,7,1.740850,0.000838,1.749884,0.002294,0.009033
150,2019-06-03 13:31:54.342665,tune bb,7,1.741971,0.000821,1.750476,0.002151,0.008505


In [355]:
# df_trial_oof2.to_pickle('../trial/oof2.pkl')

In [385]:
idx='mean'
df_test_pred = df_test_oof2
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_oof2_{}.csv'.format(idx), index=False)

In [387]:
df_test_pred.drop(columns=['index']).tail()

,27,452,722,1198,1196,469,562
2619,5.458158,5.503703,5.627529,5.479102,5.711849,5.409761,5.492808
2620,7.041155,6.692170,7.146205,6.918566,7.050169,6.743031,6.890598
2621,4.156903,4.552874,4.258360,4.317536,4.486172,4.397887,4.421894
2622,2.160095,2.157123,2.256340,2.186675,2.259750,1.930149,2.029919
2623,9.395880,8.387976,9.255499,9.739490,9.758149,9.825698,9.750566
